# Assignment 10: Join and Merge in SQL

## Introduction

For this assignment, you will continue working with SQL databases. This time, you should use Python to write the SQL questies. If possible, please submit your answers in HTML format. The data and questions are listed below.


In [ ]:
DROP TABLE IF EXISTS directors CASCADE;
DROP TABLE IF EXISTS movies CASCADE;
CREATE TABLE directors (
    director_id SERIAL PRIMARY KEY,
    director_name VARCHAR(100),
    country VARCHAR(50),
    birth_year INT,
    awards INT
);

CREATE TABLE movies (
    movie_id SERIAL PRIMARY KEY,
    title VARCHAR(100),
    director_id INT,
    release_year INT,
    box_office DECIMAL(12,2),
    rating DECIMAL(3,1),
    FOREIGN KEY (director_id) REFERENCES directors(director_id)
);

INSERT INTO directors (director_name, country, birth_year, awards) VALUES
('Christopher Nolan', 'UK', 1970, 5),
('Greta Gerwig', 'USA', 1983, 3),
('Bong Joon-ho', 'South Korea', 1969, 4),
('Sofia Coppola', 'USA', 1971, 2),
('Pedro Almodóvar', 'Spain', 1949, 6),
('Agnès Varda', 'France', 1928, 4);

INSERT INTO movies (title, director_id, release_year, box_office, rating) VALUES
('Oppenheimer', 1, 2023, 950000000.00, 8.5),
('Barbie', 2, 2023, 1440000000.00, 7.0),
('Parasite', 3, 2019, 258773645.00, 8.9),
('Lost in Translation', 4, 2003, 119723856.00, 7.7),
('Pain and Glory', 5, 2019, 38219573.00, 7.5),
('Faces Places', 6, 2017, 903996.00, 7.9),
('Inception', 1, 2010, 836836967.00, 8.8),
('Lady Bird', 2, 2017, 78965367.00, 7.4);

In [2]:
# Please import the necessary packages and establish the connection to the database
import psycopg2
from psycopg2 import sql
import pandas as pd

In [ ]:
#This is all theoretical given that I can't access my database, so this will not run, but the code is what I would do

conn = psycopg2.connect(
    dbname="dbname",
    user="username",
    password="password",
    host="hostname",
    port="portname"
)
cur = conn.cursor()

# Drop tables if they exist and create new ones
cur.execute("""
DROP TABLE IF EXISTS directors CASCADE;
DROP TABLE IF EXISTS movies CASCADE;

CREATE TABLE directors (
    director_id SERIAL PRIMARY KEY,
    director_name VARCHAR(100),
    country VARCHAR(50),
    birth_year INT,
    awards INT
);

CREATE TABLE movies (
    movie_id SERIAL PRIMARY KEY,
    title VARCHAR(100),
    director_id INT,
    release_year INT,
    box_office DECIMAL(12,2),
    rating DECIMAL(3,1),
    FOREIGN KEY (director_id) REFERENCES directors(director_id)
);

INSERT INTO directors (director_name, country, birth_year, awards) VALUES
('Christopher Nolan', 'UK', 1970, 5),
('Greta Gerwig', 'USA', 1983, 3),
('Bong Joon-ho', 'South Korea', 1969, 4),
('Sofia Coppola', 'USA', 1971, 2),
('Pedro Almodóvar', 'Spain', 1949, 6),
('Agnès Varda', 'France', 1928, 4);

INSERT INTO movies (title, director_id, release_year, box_office, rating) VALUES
('Oppenheimer', 1, 2023, 950000000.00, 8.5),
('Barbie', 2, 2023, 1440000000.00, 7.0),
('Parasite', 3, 2019, 258773645.00, 8.9),
('Lost in Translation', 4, 2003, 119723856.00, 7.7),
('Pain and Glory', 5, 2019, 38219573.00, 7.5),
('Faces Places', 6, 2017, 903996.00, 7.9),
('Inception', 1, 2010, 836836967.00, 8.8),
('Lady Bird', 2, 2017, 78965367.00, 7.4);
""")


conn.commit()

# Close the cursor and connection
cur.close()
conn.close()

#### As you know, I have had issues with this situation, and so I will not be able to establish the proper connection, but only the packages and writing the code.

1. Write a query using `INNER JOIN` to display the movie title, director name, and box office earnings for all movies, ordered by box office earnings in descending order

In [ ]:
# Assuming `cur` is the cursor object from the database connection
query = """
SELECT movies.title, directors.director_name, movies.box_office
FROM movies
INNER JOIN directors ON movies.director_id = directors.director_id
ORDER BY movies.box_office DESC;
"""

cur.execute(query)
results = cur.fetchall()

# Display the results
for row in results:
    print(row)

2. Using a `LEFT JOIN`, find all directors and count the number of movies they have directed.

In [ ]:
query = """
SELECT directors.director_name, COUNT(movies.movie_id) AS movie_count
FROM directors
LEFT JOIN movies ON directors.director_id = movies.director_id
GROUP BY directors.director_name;
"""

cur.execute(query)
results = cur.fetchall()


for row in results:
    print(row)

3. Write a `SELF JOIN` query to compare the ratings of movies by the same director. Show only pairs where the second movie has a higher rating than the first.

In [ ]:
query = """
SELECT m1.title AS movie1, m2.title AS movie2, m1.rating AS rating1, m2.rating AS rating2, directors.director_name
FROM movies AS m1
JOIN movies AS m2 ON m1.director_id = m2.director_id AND m1.movie_id < m2.movie_id
JOIN directors ON m1.director_id = directors.director_id
WHERE m2.rating > m1.rating;
"""

cur.execute(query)
results = cur.fetchall()


for row in results:
    print(row)


4. Using appropriate joins, find directors who have made movies with above-average box office earnings (compared to all movies in the database).

In [ ]:
query = """
WITH average_box_office AS (
    SELECT AVG(box_office) AS avg_box_office
    FROM movies
)
SELECT directors.director_name, movies.title, movies.box_office
FROM movies
JOIN directors ON movies.director_id = directors.director_id
JOIN average_box_office ON movies.box_office > average_box_office.avg_box_office;
"""

cur.execute(query)
results = cur.fetchall()


for row in results:
    print(row)


5. Create a query using `CROSS JOIN` to show all possible combinations of directors and movies, even if they did not direct them. Limit the output to 10 rows.


In [ ]:
query = """
SELECT directors.director_name, movies.title
FROM directors
CROSS JOIN movies
LIMIT 10;
"""

cur.execute(query)
results = cur.fetchall()


for row in results:
    print(row)


6. Write a query that uses `UNION` to create a list of all director names and movie titles in a single column. Label the column `name` and include a column (called `type`) indicating if it is a director or movie. Order the results by type and name.

Hint: Use `'Director'` and `'Movie'` as the labels for the second column. Please also use `as type` to rename the column (example `director_name as name, 'Director' as type`).

In [ ]:
query = """
SELECT director_name AS name, 'Director' AS type
FROM directors
UNION
SELECT title AS name, 'Movie' AS type
FROM movies
ORDER BY type, name;
"""

cur.execute(query)
results = cur.fetchall()


for row in results:
    print(row)


7. Using appropriate joins, find the director with the highest average movie rating. Show only the row with the director's name, average rating, and number of movies.

In [ ]:
query = """
SELECT directors.director_name, AVG(movies.rating) AS average_rating, COUNT(movies.movie_id) AS movie_count
FROM directors
JOIN movies ON directors.director_id = movies.director_id
GROUP BY directors.director_name
ORDER BY average_rating DESC
LIMIT 1;
"""

cur.execute(query)
results = cur.fetchall()


for row in results:
    print(row)


8. Create a query using `LEFT JOIN` and `IS NULL` to find whether there are directors who have not directed any movies.

In [ ]:
query = """
SELECT directors.director_name
FROM directors
LEFT JOIN movies ON directors.director_id = movies.director_id
WHERE movies.movie_id IS NULL;
"""

cur.execute(query)
results = cur.fetchall()


for row in results:
    print(row)


9. Using appropriate joins, find pairs of movies released in the same year, along with their directors' names. Please do not match a movie with itself. 

In [ ]:
query = """
SELECT m1.title AS movie1, m2.title AS movie2, d1.director_name AS director1, d2.director_name AS director2, m1.release_year
FROM movies AS m1
JOIN movies AS m2 ON m1.release_year = m2.release_year AND m1.movie_id < m2.movie_id
JOIN directors AS d1 ON m1.director_id = d1.director_id
JOIN directors AS d2 ON m2.director_id = d2.director_id;
"""

cur.execute(query)
results = cur.fetchall()


for row in results:
    print(row)


10. Show the age of each director when they released their movies. Create a column entitled `age_at_release` in your output. Order the results by the director's name and the movie's release year.

In [ ]:
query = """
SELECT directors.director_name, movies.title, movies.release_year,
       (movies.release_year - directors.birth_year) AS age_at_release
FROM directors
JOIN movies ON directors.director_id = movies.director_id
ORDER BY directors.director_name, movies.release_year;
"""

cur.execute(query)
results = cur.fetchall()

# Display the results
for row in results:
    print(row)


Good luck! 😃